In [13]:
import os
import numpy as np
import pandas as pd
import numpy.random as rnd
import seaborn as sns
from matplotlib import animation
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sys import platform
import inspect
import arviz as az

az.style.use('arviz-darkgrid')
az.rcParams['plot.max_subplots'] = 50

In [14]:
# Read the data
#set current working directory to where this file is saved
# thisdir = globals()['_dh'][0] + "\\" 
# os.chdir(thisdir)

# if platform == "darwin": 
#     # OS X / Mac
#     os.environ["PATH"] += os.pathsep + "/usr/local/bin"
# elif platform == "win32":
#     # Windows
#     pass

#path = "/Users/donaldbrown/Dropbox/department/Classes/DS6014/CourseraBayesianML/Week4MCMC/"
file = "2krolls_plus_data.csv"

data = pd.read_csv(file)
# drop adv due to potential dependence
#data = data.drop("adv", axis=1)
data.head()

,id,roll,adv,stat,basis,type
0,0,5,none,STR,dexterity,save
1,1,11,none,DEX,dexterity,save
2,2,20,advantage,DEX,constitution,check
3,3,7,none,DEX,strength,check
4,4,3,none,WIS,melee,attack


In [15]:
file2 = "TravisRollsDataset.csv"

data2 = pd.read_csv(file2)
data2.head()

,id,season,episode,roll,mod,basis,type,adv,critical,stat
0,2,3,1,13,0,wisdom,save,none,none,WIS
1,3,3,1,17,0,investigation,check,none,none,INT
2,4,3,1,4,2,dexterity,save,none,none,DEX
3,6,3,2,16,7,melee,attack,none,none,ATTACK
4,7,3,2,16,7,melee,attack,advantage,none,ATTACK


In [16]:
# create a column to delineate between simulated data and Travis' rolls
# 0 is a Travis roll
#deter = [0] * len(data2)
data2["deter"] = 0
# create bins for the rolls
data2["rollbins"] = pd.cut(data2.roll, bins=[0, 8, 15, 20], labels=[0,1,2])

# 1 is a generated roll
#deter = [1] * len(data)
data["deter"] = 1
data["rollbins"] = pd.cut(data.roll, bins=[0, 8, 15, 20], labels=[0,1,2])

In [19]:
def assess_important(x):
    if x['adv']=="disadvantage" or x['basis'] in ["melee", "ranged", "dexterity", "strength"]:
        return 1
    else:
        return 0
data['important'] = data.apply(assess_important, axis=1)
data2['important'] = data2.apply(assess_important, axis=1)

In [35]:
# get dummies for generated data
data_dums1 = pd.get_dummies(data[["adv", "stat", "important"]])
# add back roll column
data_dums1["roll"] = data.roll

# get dummies for travis data
data_dums2 = pd.get_dummies(data2[["adv", "stat", "important"]])
# add back roll column
data_dums2["roll"] = data2.roll

# rodeo_dums2 = rodeo_dums[["adv_advantage", "adv_disadvantage", "stat_ATTACK"]]
# rodeo_dums

In [27]:
# logistic estimate of the posterior distribution for TRAVIS DATA

with pm.Model() as travis_log:
        Y = data2["deter"]
        pm.glm.GLM.from_formula('Y ~ adv_advantage + adv_disadvantage + adv_none + stat_ATTACK + roll',\
                                data_dums2, family=pm.glm.families.Binomial())
        trace = pm.sample(1000, tune=1000, cores=1, progressbar=True, target_accept=.99) #, init='adapt_diag', cores =2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [roll, stat_ATTACK, adv_none, adv_disadvantage, adv_advantage, Intercept]
Sampling chain 1, 0 divergences:   6%|▌         | 113/2000 [00:00<00:16, 113.38it/s]


ValueError: Mass matrix contains zeros on the diagonal. 
The derivative of RV `roll`.ravel()[0] is zero.
The derivative of RV `stat_ATTACK`.ravel()[0] is zero.
The derivative of RV `adv_none`.ravel()[0] is zero.
The derivative of RV `adv_disadvantage`.ravel()[0] is zero.
The derivative of RV `adv_advantage`.ravel()[0] is zero.

In [44]:
# logistic estimate of the posterior distribution for GENERATED DATA

# ARE THERE ANY ATTACK ROLLS IN THIS SET???

with pm.Model() as gen_log:
        Y = data["deter"]
        pm.glm.GLM.from_formula('Y ~ adv_advantage + adv_disadvantage + adv_none + stat_ATTACK',\
                                data_dums1, family=pm.glm.families.Binomial())
        trace = pm.sample(1000, tune=1000, cores=1, progressbar=True, target_accept=.99) #, init='adapt_diag', cores =2)

PatsyError: Error evaluating factor: NameError: name 'stat_ATTACK' is not defined
    Y ~ adv_advantage + adv_disadvantage + adv_none + stat_ATTACK
                                                      ^^^^^^^^^^^

In [5]:
# # make list of column names
# 
# col_names.remove("Unnamed: 0")
# col_names.remove("basis_type")
# col_names

In [6]:
# # pick out shared columns
# col_names = list(data.columns)
# merge1 = data[col_names]
# merge2 = data2[col_names]
# # 
# # merge the datasets
# total_data_rodeo = pd.concat([merge1, merge2], axis=0).reset_index()

# # shuffle the data 3x in replicable way
# rodeo = total_data_rodeo.sample(frac=1, random_state=1234567).reset_index(drop=True)
# rodeo = rodeo.sample(frac=1, random_state=57389).reset_index(drop=True)
# rodeo = rodeo.sample(frac=1, random_state=98754).reset_index(drop=True)
# rodeo = rodeo[col_names]

In [7]:
# # put rolls into bins
# rodeo["rollbins"] = pd.cut(rodeo.roll, bins=[0, 8, 15, 20], labels=[0,1,2])

# def assess_important(x):
#     if x['adv']=="disadvantage" or x['basis'] in ["melee", "ranged", "dexterity", "strength"]:
#         return 1
#     else:
#         return 0
# rodeo['important'] = rodeo.apply(assess_important, axis=1)

In [8]:
# series_vals = [str(x) for x in rodeo['roll'].values.tolist()]
# cats = [str(x) for x in list(range(1,21))]

# rodeo['roll'] = pd.Categorical(series_vals, 
#                                    categories= cats,
#                                    ordered=False)
# rodeo["basis_type"] = rodeo["basis"].astype(str) + "_" + rodeo["type"].astype(str)

# rodeo = rodeo.drop(["basis","type","id"], axis=1)

In [11]:
# rodeo_dums = pd.get_dummies(rodeo[["adv", "stat", "important"]])
# rodeo_dums2 = rodeo_dums[["adv_advantage", "adv_disadvantage", "stat_ATTACK"]]
# # rodeo_dums

,important,adv_advantage,adv_disadvantage,adv_none,stat_ATTACK,stat_BASE,stat_CHA,stat_CON,stat_DEX,stat_INT,stat_STR,stat_WIS
0,1,0,0,1,0,0,0,0,1,0,0,0
1,1,0,0,1,0,0,0,0,1,0,0,0
2,1,0,1,0,0,0,0,0,0,0,1,0
3,1,0,0,1,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2165,1,0,0,1,0,0,1,0,0,0,0,0
2166,0,0,0,1,0,0,0,0,0,0,1,0
2167,1,0,0,1,0,1,0,0,0,0,0,0
2168,1,0,0,1,0,0,0,0,0,0,0,1


In [12]:
## feature selected model

# logistic estimate of the posterior distribution

with pm.Model() as log_model:
        Y = rodeo["deter"]
        pm.glm.GLM.from_formula('Y ~ adv_advantage + adv_disadvantage + adv_none + stat_ATTACK',\
                                rodeo_dums, family=pm.glm.families.Binomial())
        trace = pm.sample(1000, tune=5000, cores=1, progressbar=True, target_accept=.99) #, init='adapt_diag', cores =2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [stat_ATTACK, adv_none, adv_disadvantage, adv_advantage, Intercept]
Sampling chain 1, 0 divergences:  82%|████████▏ | 4894/6000 [17:30<03:57,  4.66it/s]
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks


$Determination = \beta_0 + \beta_1 *$adv_advantage + $\beta_2 *$adv_disadvantage + $\beta_3 *$adv_none + $\beta_4 *$stat_ATTACK

In [ ]:
determination = rodeo["deter"]
rolls = rodeo["roll"]
rollbins = rodeo["rollbins"]

In [ ]:
rodeo_dums.columns

In [ ]:
# # Build the model with priors and run the sampling
# with pm.Model() as chd_model:
    
#     # Intercept term & prior
#     beta0 = pm.Normal("beta0", mu=0, sd=1)
#     # Beta coefficients for predictor variables & priors
#     beta = pm.MvNormal("beta", mu=np.zeros(k), cov=np.eye(k), shape=k)
    
    
#     # Calculate the logit 
#     mu = beta0 + pm.math.dot(X, beta)
#     theta = pm.Deterministic("theta",  pm.invlogit(mu))
#     # Pass the logits to a Bernoulli outcome, with the observed data
#     y_hat = pm.Bernoulli("y_hat", p=theta, observed=y) 
    
#     # Sample
#     trace_main = pm.sample(10000, cores=1, tune=1000, init='adapt_diag', progressbar=True)


In [ ]:
## https://docs.pymc.io/notebooks/GLM-linear.html

import pymc3 as pm
from pymc3 import  *

#copy the x and y values into new variables
ycol = rodeo.deter.copy()
#don't include die outcome column
xcols = rodeo_dums.copy() 

#collect count values
xcolumns = list(xcols.columns)
coeff_count = len(xcolumns)
die_side_count = rodeo.rollbins.nunique()

In [ ]:
with pm.Model() as hierarchical_model:

    # Priors for the model parameters
    # Gaussians for the means of the priors of the random intercepts and slopes
    mus = []
    for i in range(coeff_count):
        mus.append(pm.Normal("mu_"+str(i), mu=0., sd=1e5))
    
    # Half-Cauchy for the standard deviations of the priors 
    # of the random intercepts and slopes
    sigmas = []
    for i in range(coeff_count):
        sigmas.append(pm.HalfCauchy("sigma_"+str(i), 1e5))

    # Gaussian priors for random intercepts and slopes
    priors = []
    for i in range(coeff_count):
        priors.append(pm.Normal("prior_"+str(i), mu=mus[i], 
                            sd=sigmas[i], shape=die_side_count))
    
    # Linear model, initialize w intercept
    mu = priors[0][rollbins] 
    
    for die in range(1,coeff_count):
        mu += priors[die][rollbins] * xcols.iloc[:, die]
    
    # Transform outcome to probability
    theta = pm.Deterministic('theta', pm.invlogit(mu))
    y_hat = pm.Bernoulli('y_hat', p=theta, observed=ycol)
    
    # Sample the posterior
    hierarchical_trace = pm.sample(draws=500, #1000
                            step=pm.NUTS(target_accept=0.99), 
                            tune=2000) #5000

# with Model() as model:
#     # specify glm and pass in data. The resulting linear model, its likelihood and
#     # and all its parameters are automatically added to our model.
#     glm.linear.LinearComponent(x=xcols, y=ycol, intercept=True)
#     trace = sample(3000, cores=2) # draw 3000 posterior samples using NUTS sampling

In [ ]:
# Trace plots
with hierarchical_model:
    az.plot_trace(hierarchical_trace)

In [ ]:
# Results in table
with hierarchical_model:
    main_idata = az.from_pymc3(hierarchical_trace)
    
azsum_df = az.summary(main_idata, round_to=2) #var_names=xcols.columns, 
azsum_df.sort_values(by=["r_hat"])

In [ ]:
# Forest plots
plt.figure(figsize=(6,14))
with hierarchical_model:
    pm.forestplot(hierarchical_trace); #var_names=xcols.columns, , combined=True